# MCP Genie Tool-Calling Agent with Databricks Claude Sonnet 4

This notebook demonstrates how to create a powerful tool-calling agent using Databricks' Claude Sonnet 4 endpoint with the MCP (Model Context Protocol) Genie server. The agent can query structured data tables through Genie spaces and perform complex reasoning tasks.

## Overview

The MCP Genie server allows you to:
- Query Genie spaces to get insights from structured data tables
- Access structured data through natural language queries
- Integrate data insights into conversational AI workflows

## Prerequisites

- Databricks workspace with access to **Claude Sonnet 4 endpoint**
- MCP Genie server access (Beta)
- **OAuth authentication** (Service Principal with OAuth credentials)
- Python 3.12 or above
- Required Python packages (see installation section below)

> **⚠️ Important**: This notebook uses OAuth authentication exclusively and is configured for **Claude Sonnet 4** by default. You must set up a Service Principal with OAuth credentials to use this agent.

## Installation and Setup

First, let's install the required dependencies:

In [1]:
# Install all required packages from requirements.txt
%pip install -r requirements.txt

print("✅ All required packages installed from requirements.txt!")
print("📋 Next: Set up OAuth authentication using the .env file")


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
✅ All required packages installed from requirements.txt!
📋 Next: Set up OAuth authentication using the .env file


## Import Required Libraries

Import all necessary libraries for the MCP tool-calling agent:

In [ ]:
import asyncio
import mlflow
import os
import json
from uuid import uuid4
from pydantic import BaseModel, create_model
from typing import Annotated, Any, Generator, List, Optional, Sequence, TypedDict, Union

from databricks_langchain import (
    ChatDatabricks, 
    UCFunctionToolkit, 
    VectorSearchRetrieverTool
)
from databricks_mcp import DatabricksOAuthClientProvider, DatabricksMCPClient
from databricks.sdk import WorkspaceClient
from langchain_core.language_models import LanguageModelLike
from langchain_core.runnables import RunnableConfig, RunnableLambda
from langchain_core.messages import (
    AIMessage, 
    AIMessageChunk, 
    BaseMessage, 
    convert_to_openai_messages
)
from langchain_core.tools import BaseTool, tool
from langgraph.graph import END, StateGraph
from langgraph.graph.message import add_messages
from langgraph.graph.state import CompiledStateGraph
from langgraph.prebuilt.tool_node import ToolNode
from mcp import ClientSession
from mcp.client.streamable_http import streamablehttp_client as connect
from mlflow.entities import SpanType
from mlflow.pyfunc import ResponsesAgent
from mlflow.types.responses import (
    ResponsesAgentRequest, 
    ResponsesAgentResponse, 
    ResponsesAgentStreamEvent
)

# Import configuration
from config import config, validate_oauth_setup

## Configuration

Configure the LLM endpoint and system prompt. We'll use Databricks' **Claude Sonnet 4** endpoint:

In [ ]:
# Configuration loaded from config.py
# LLM and system prompt configuration - defaults to Claude Sonnet 4
llm = ChatDatabricks(endpoint=config.llm_endpoint_name)
system_prompt = config.system_prompt

print("✅ Configuration loaded from config.py")
print(f"🔗 LLM Endpoint: {config.llm_endpoint_name} (Claude Sonnet 4)")
print(f"🌐 Workspace: {config.databricks_host}")
print(f"🗂️ Genie Space ID: {config.genie_space_id}")
print(f"📡 MCP Server URL: {config.genie_server_url}")

## Agent State Definition

Define the state structure for our LangGraph agent:

In [ ]:
# Configure MCP tools and agent workflow
class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], add_messages]
    custom_inputs: Optional[dict[str, Any]]

## MCP Genie Client Setup

Set up the MCP client to connect to the Genie server. **Note**: You'll need to provide the Genie space ID and authentication details.

In [ ]:
# Configuration is now managed in config.py
# This provides better security and organization

print("✅ Configuration externalized to config.py")
print("📁 All settings are now managed in the config file")
print("🔒 OAuth credentials are loaded from environment variables")
print("\n💡 To configure your credentials, choose one of these methods:")
print()
print("Method 1: Environment Variables")
print('export DATABRICKS_HOST="https://your-workspace.cloud.databricks.com"')
print('export DATABRICKS_CLIENT_ID="your-service-principal-client-id"')
print('export DATABRICKS_CLIENT_SECRET="your-oauth-secret"')
print('export GENIE_SPACE_ID="your-genie-space-id"')
print()
print("Method 2: Programmatic Setup (see next cell)")
print()
print("Method 3: .databrickscfg file")
print("[default]")
print("host = https://your-workspace.cloud.databricks.com")
print("client_id = your-service-principal-client-id")
print("client_secret = your-oauth-secret")

## OAuth Authentication Setup

**This notebook requires OAuth authentication with Service Principal credentials.**

### Step 1: Create Service Principal (One-time setup)

1. **In your Databricks workspace**:
   - Go to **Settings** → **Identity and access** → **Service principals**
   - Click **Add service principal**
   - Enter name: `MCP-Genie-Agent`
   - Click **Add**

2. **Generate OAuth credentials**:
   - Select your service principal
   - Go to **OAuth secrets** tab
   - Click **Generate secret**
   - Set lifetime (max 730 days, recommended: 365 days)
   - **⚠️ Copy the Client ID and Client Secret** (shown only once!)

3. **Assign workspace permissions**:
   - Add the service principal to your workspace
   - Grant necessary permissions for Genie space access

### Step 2: Configure OAuth Authentication

Choose one of the following authentication methods:

In [ ]:
# Programmatic Configuration Setup
# Use this if you prefer to set credentials directly in the notebook

# Uncomment and configure these with your actual values:
# config.set_oauth_credentials(
#     client_id="your-service-principal-client-id",
#     client_secret="your-oauth-secret", 
#     workspace_host="your-workspace.cloud.databricks.com"
# )
# config.set_genie_space_id("your-genie-space-id")

print("🔧 Programmatic configuration helper available")
print("📝 Uncomment and modify the config.set_oauth_credentials() call above")
print("⚠️  Remember: Don't commit actual credentials to version control!")

## OAuth Authentication Test

Run this cell to verify your OAuth setup is working correctly:

In [ ]:
# Configuration Validation
# This uses the config.py validation functions

def test_oauth_configuration():
    """Test OAuth authentication configuration using config.py."""
    print("🧪 Testing OAuth Configuration...")
    print("=" * 50)
    
    # Check for conflicting authentication methods
    import os
    if "DATABRICKS_TOKEN" in os.environ:
        print("⚠️  Warning: DATABRICKS_TOKEN is set, which conflicts with OAuth!")
        print("   Unsetting DATABRICKS_TOKEN for this session...")
        del os.environ["DATABRICKS_TOKEN"]
    
    # Use the validation function from config.py
    is_valid = validate_oauth_setup()
    
    if not is_valid:
        return False
    
    # Test Databricks WorkspaceClient initialization
    try:
        print("\n🔄 Testing Databricks WorkspaceClient...")
        workspace_client = WorkspaceClient()
        
        # Test authentication by getting current user
        current_user = workspace_client.current_user.me()
        print(f"✅ Authentication successful!")
        print(f"   User: {current_user.user_name}")
        print(f"   Active: {current_user.active}")
        
        return True
        
    except Exception as e:
        print(f"❌ OAuth authentication failed: {e}")
        print("\n🔧 Troubleshooting steps:")
        print("1. Verify Client ID and Client Secret are correct")
        print("2. Check that the service principal is assigned to the workspace")
        print("3. Ensure the OAuth secret hasn't expired")
        print("4. Verify the workspace hostname is correct")
        return False

# Run the test
oauth_success = test_oauth_configuration()

if oauth_success:
    print(f"\n🎉 OAuth configuration is ready!")
    print(f"🚀 You can now proceed to test the MCP Genie agent")
else:
    print(f"\n⚠️  Please fix the OAuth configuration before proceeding")

## MCP Client Initialization

Initialize the MCP client to connect to the Genie server:

In [ ]:
async def create_mcp_client():
    """
    Create and initialize MCP client for Genie server connection using OAuth.
    Uses configuration from config.py
    """
    try:
        print("🔄 Initializing MCP client with OAuth authentication...")

        # Initialize Databricks workspace client with OAuth
        # The WorkspaceClient will automatically use OAuth credentials from:
        # 1. Environment variables (DATABRICKS_CLIENT_ID, DATABRICKS_CLIENT_SECRET)
        # 2. Configuration file (~/.databrickscfg)
        # 3. Databricks CLI profile
        workspace_client = WorkspaceClient()

        print("✅ Databricks workspace client initialized")

        # Create MCP client with the workspace client
        # Uses the genie_server_url from config.py
        mcp_client = DatabricksMCPClient(
            server_url=config.genie_server_url,
            workspace_client=workspace_client
        )

        print(f"✅ MCP client created successfully")
        print(f"🔗 Connected to: {config.genie_server_url}")
        return mcp_client

    except Exception as e:
        print(f"❌ Error creating MCP client: {e}")
        print("\n🔧 Troubleshooting:")
        print("1. Ensure OAuth credentials are properly configured in config.py")
        print("2. Verify WORKSPACE_HOSTNAME and GENIE_SPACE_ID are correct")
        print("3. Check that the service principal has access to the Genie space")
        print("4. Confirm the OAuth secret hasn't expired")
        print("5. Make sure databricks-mcp package is properly installed")
        return None

# OAuth authentication will be used when this function is called

## Tool Creation from MCP Server

Create LangChain tools from the MCP server capabilities:

In [ ]:
async def get_mcp_tools():
    """
    Retrieve available tools from the MCP Genie server.
    """
    mcp_client = await create_mcp_client()
    
    if mcp_client is None:
        print("Failed to create MCP client - returning empty tools list")
        return []
    
    try:
        # Get available tools from MCP server using the private async method
        # Note: Using private method due to asyncio.run() conflicts in Jupyter
        tools = await mcp_client._get_tools_async()
        
        print(f"Found {len(tools)} tools from MCP Genie server:")
        for tool_def in tools:
            print(f"  - {tool_def.name}: {tool_def.description[:100]}...")
        
        # Convert MCP tools to LangChain tools using BaseTool class
        from langchain_core.tools import BaseTool
        from typing import ClassVar, Any
        import concurrent.futures
        
        langchain_tools = []
        
        # Create tools using BaseTool class with proper type annotations
        for mcp_tool in tools:
            class DynamicMCPTool(BaseTool):
                name: str = mcp_tool.name
                description: str = mcp_tool.description
                # Use ClassVar to indicate these are not Pydantic fields
                tool_def: ClassVar[Any] = mcp_tool
                client: ClassVar[Any] = mcp_client
                
                class Config:
                    arbitrary_types_allowed = True
                
                async def _arun(self, query: str = "", **kwargs) -> str:
                    """Async tool execution."""
                    try:
                        # Merge query parameter with other kwargs
                        params = {"query": query, **kwargs}
                        # Use the private async method that works in Jupyter
                        result = await self.client._call_tools_async(self.tool_def.name, params)
                        
                        # Extract text content from the response
                        if hasattr(result, 'content') and result.content:
                            if isinstance(result.content, list) and len(result.content) > 0:
                                if hasattr(result.content[0], 'text'):
                                    return result.content[0].text
                        
                        return str(result)
                    except Exception as e:
                        return f"Error calling tool {self.tool_def.name}: {e}"
                
                def _run(self, query: str = "", **kwargs) -> str:
                    """Sync tool execution using thread executor."""
                    def run_in_new_loop():
                        # Create a new event loop for this thread
                        import asyncio
                        new_loop = asyncio.new_event_loop()
                        asyncio.set_event_loop(new_loop)
                        try:
                            return new_loop.run_until_complete(self._arun(query, **kwargs))
                        finally:
                            new_loop.close()
                    
                    # Run in a separate thread to avoid event loop conflicts
                    with concurrent.futures.ThreadPoolExecutor() as executor:
                        future = executor.submit(run_in_new_loop)
                        return future.result(timeout=60)  # 60 second timeout
            
            # Create an instance of the tool
            tool_instance = DynamicMCPTool()
            langchain_tools.append(tool_instance)
        
        print(f"✅ Successfully created {len(langchain_tools)} LangChain tools")
        return langchain_tools
        
    except Exception as e:
        print(f"Error retrieving MCP tools: {e}")
        import traceback
        traceback.print_exc()
        return []

# This will be called when authentication is ready

## Agent Node Functions

Define the core functions that will be used in our LangGraph workflow:

In [ ]:
def should_continue(state: AgentState) -> str:
    """
    Determine whether to continue the conversation or end it.
    """
    messages = state["messages"]
    last_message = messages[-1]
    
    # If the LLM makes a tool call, continue to the tool node
    if last_message.tool_calls:
        return "tools"
    # Otherwise, end the conversation
    return END


def call_model(state: AgentState) -> dict:
    """
    Call the language model with the current state.
    """
    messages = state["messages"]
    
    # Add system prompt as first message if not present
    if not messages or messages[0].content != system_prompt:
        system_message = {"role": "system", "content": system_prompt}
        messages = [system_message] + list(messages)
    
    # Call the LLM
    response = llm.invoke(messages)
    
    # Return the response in the expected format
    return {"messages": [response]}

## Agent Graph Construction

Build the LangGraph workflow that orchestrates the conversation flow:

In [ ]:
async def create_agent() -> CompiledStateGraph:
    """
    Create and compile the LangGraph agent.
    """
    # Get MCP tools
    tools = await get_mcp_tools()
    
    # Create tool node
    tool_node = ToolNode(tools)
    
    # Bind tools to the LLM
    llm_with_tools = llm.bind_tools(tools)
    
    # Update call_model to use LLM with tools
    def call_model_with_tools(state: AgentState) -> dict:
        messages = state["messages"]
        
        # Add system prompt if not present
        if not messages or not any(msg.content == system_prompt for msg in messages if hasattr(msg, 'content')):
            from langchain_core.messages import SystemMessage
            system_message = SystemMessage(content=system_prompt)
            messages = [system_message] + list(messages)
        
        # Call the LLM with tools
        response = llm_with_tools.invoke(messages)
        return {"messages": [response]}
    
    # Define the graph
    workflow = StateGraph(AgentState)
    
    # Add nodes
    workflow.add_node("agent", call_model_with_tools)
    workflow.add_node("tools", tool_node)
    
    # Set entry point
    workflow.set_entry_point("agent")
    
    # Add conditional edges
    workflow.add_conditional_edges(
        "agent",
        should_continue,
        {
            "tools": "tools",
            END: END,
        },
    )
    
    # Add edge from tools back to agent
    workflow.add_edge("tools", "agent")
    
    # Compile the graph
    app = workflow.compile()
    
    return app

## MLflow Integration

Create an MLflow-compatible agent wrapper for tracking and deployment:

In [ ]:
class MCPGenieAgent(ResponsesAgent):
    """
    MLflow-compatible wrapper for the MCP Genie agent.
    """
    
    def __init__(self, agent_graph: CompiledStateGraph):
        self.agent = agent_graph
    
    def predict(self, context, model_input, params=None):
        """
        Required predict method for MLflow compatibility.
        """
        # Convert model_input to the expected format
        if isinstance(model_input, str):
            # Simple string input
            request = ResponsesAgentRequest(
                messages=[{"role": "user", "content": model_input}]
            )
        else:
            # Assume it's already in the right format
            request = model_input
            
        return self.invoke(request)
    
    def invoke(self, request: ResponsesAgentRequest) -> ResponsesAgentResponse:
        """
        Process a single request and return a response.
        """
        from langchain_core.messages import HumanMessage
        
        # Convert request to agent state
        if hasattr(request, 'messages') and request.messages:
            content = request.messages[-1].get('content', str(request.messages[-1]))
        else:
            content = str(request)
            
        messages = [HumanMessage(content=content)]
        state = AgentState(messages=messages, custom_inputs=None)
        
        # Run the agent synchronously (since we're in an async context already)
        import nest_asyncio
        nest_asyncio.apply()
        
        try:
            # Use asyncio.create_task for proper async handling in Jupyter
            import asyncio
            if asyncio.get_event_loop().is_running():
                # We're in a running event loop (Jupyter), use different approach
                result = asyncio.run_coroutine_threadsafe(
                    self._async_invoke(state), 
                    asyncio.new_event_loop()
                ).result()
            else:
                result = asyncio.run(self._async_invoke(state))
        except Exception as e:
            print(f"Error in agent invocation: {e}")
            result = {"messages": [HumanMessage(content=f"Error: {e}")]}
        
        # Extract response
        response_content = result["messages"][-1].content
        
        return ResponsesAgentResponse(
            id=str(uuid4()),
            created=int(asyncio.get_event_loop().time()),
            object="agent.completion",
            model=config.llm_endpoint_name,
            choices=[
                {
                    "index": 0,
                    "message": {
                        "role": "assistant",
                        "content": response_content
                    },
                    "finish_reason": "stop"
                }
            ],
            usage={
                "prompt_tokens": 0,  # Would need to implement token counting
                "completion_tokens": 0,
                "total_tokens": 0
            }
        )
    
    async def _async_invoke(self, state: AgentState):
        """Helper method for async agent invocation."""
        agent_graph = await create_agent()
        return agent_graph.invoke(state)
    
    def stream(self, request: ResponsesAgentRequest) -> Generator[ResponsesAgentStreamEvent, None, None]:
        """
        Stream responses for real-time interaction.
        """
        # For now, just yield the complete response
        response = self.invoke(request)
        
        yield ResponsesAgentStreamEvent(
            id=response.id,
            created=response.created,
            object="agent.completion.chunk",
            model=response.model,
            choices=[
                {
                    "index": 0,
                    "delta": {
                        "role": "assistant",
                        "content": response.choices[0]["message"]["content"]
                    },
                    "finish_reason": "stop"
                }
            ]
        )

## Testing the Agent

Test the agent with some sample queries (will work once authentication is configured):

In [ ]:
async def test_agent():
    """
    Test the MCP Genie agent with sample queries.
    """
    print("Creating MCP Genie Agent...")
    
    try:
        # Create the agent
        agent_graph = await create_agent()
        agent = MCPGenieAgent(agent_graph)
        
        print("✅ Agent created successfully!")
        
        # Test queries
        test_queries = [
            "What data sources are available in this Genie space?",
            "Can you show me a summary of the available tables?",
            "What insights can you provide about the data?"
        ]
        
        for query in test_queries:
            print(f"\n🔍 Testing query: {query}")
            
            from langchain_core.messages import HumanMessage
            state = AgentState(
                messages=[HumanMessage(content=query)],
                custom_inputs=None
            )
            
            try:
                # Direct agent graph invocation for testing
                result = agent_graph.invoke(state)
                response = result["messages"][-1].content
                print(f"📝 Response: {response}")
            except Exception as e:
                print(f"❌ Error processing query: {e}")
        
        return agent
        
    except Exception as e:
        print(f"❌ Error creating agent: {e}")
        print("Please ensure OAuth authentication is properly configured.")
        return None

# Jupyter-friendly async execution
import asyncio
import nest_asyncio
nest_asyncio.apply()

print("✅ Test function defined. Run the cell below to test the agent.")
print("🔐 Make sure OAuth authentication is configured in the .env file!")

## Interactive Usage Example

Example of how to use the agent interactively:

In [ ]:
async def interactive_session():
    """
    Run an interactive session with the MCP Genie agent.
    """
    print("Starting interactive session with MCP Genie Agent...")
    print("Type 'quit' to exit\n")
    
    # Create agent
    agent_graph = await create_agent()
    
    # Initialize conversation state
    conversation_state = AgentState(messages=[], custom_inputs=None)
    
    while True:
        user_input = input("You: ")
        
        if user_input.lower() == 'quit':
            print("Goodbye!")
            break
        
        # Add user message to conversation
        from langchain_core.messages import HumanMessage
        user_message = HumanMessage(content=user_input)
        conversation_state["messages"].append(user_message)
        
        try:
            # Get agent response
            result = agent_graph.invoke(conversation_state)
            
            # Update conversation state
            conversation_state = result
            
            # Display response
            response = result["messages"][-1].content
            print(f"Agent: {response}\n")
            
        except Exception as e:
            print(f"Error: {e}\n")

# Run interactive session once OAuth is configured
print("✅ Interactive session function defined. Run 'await interactive_session()' once OAuth authentication is configured.")
print("🔐 OAuth authentication must be configured first!")

## Quick Start Guide

Once you have OAuth authentication configured, run these cells to test the agent:

In [ ]:
# Run this cell to test the agent with sample queries
agent = await test_agent()

In [ ]:
# Interactive Chat - Run this for a conversational interface
# Type your questions and the agent will respond using Genie data
# Type 'quit' to exit

await interactive_session()

In [ ]:
# Single Query Test - Run this to test with a custom query
# Modify the query below to test specific questions

async def single_query_test(query: str):
    """Test a single query with the agent."""
    print(f"🔍 Testing query: {query}")
    
    try:
        # Create agent if not already created
        agent_graph = await create_agent()
        
        # Create state with the query
        from langchain_core.messages import HumanMessage
        state = AgentState(
            messages=[HumanMessage(content=query)],
            custom_inputs=None
        )
        
        # Get response
        result = agent_graph.invoke(state)
        response = result["messages"][-1].content
        
        print(f"📝 Response: {response}")
        return response
        
    except Exception as e:
        print(f"❌ Error: {e}")
        return None

# Example usage - modify the query as needed
query = "How many queries were executed over the past 7 days in SQL?"
response = await single_query_test(query)

## 🎉 Ready to Use!

The MCP Genie Agent is now configured and ready to query your Databricks system tables through natural language.

### Example Queries to Try:
- "How many queries were executed over the past 7 days in SQL?"
- "What are the most expensive clusters by compute cost?" 
- "Show me the top SQL queries by execution time"
- "Which users are running the most jobs?"
- "What is the total data processed in the last month?"

The agent will convert these natural language questions into SQL queries against your Databricks system tables and return the results! 🚀